## Random Forest Implementaion from Scratch

This is an implementation of random forest algorithm. The project aims to create a runnable random forest that takes input of numpy array.

In [176]:
# import
import pandas as pd
import numpy as np

from collections import Counter
from scipy.stats import mode

In [51]:
# implementation of tree node
class Node:
    '''
    Implement a decision tree node
    '''
    
    def __init__(self, feature=None, threshold=None, left=None, 
                 right=None, gain=None, value=None):
        '''
        constructor
        '''
        self.feature = feature # the selected feature on the node
        self.threshold = threshold # threshold for the data split
        self.left = left # information for left subtree
        self.right = right # information for right subtree
        self.gain = gain # gain for the split on this node
        self.value = value # target value in leaf node (only!)

In [350]:
# implement of classification tree
class Tree:
    '''
    Implement a binary ID3 decision tree
    '''
    def __init__(self, min_samples_split=5, max_depth=3,
                feature_bagging = None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        self.feature_bagging = feature_bagging # None for no feature bagging, else be number of selected feature, must be > 0 and < p
        
        
    def entropy(self, group):
        '''
        Compute the entropy of group
        
        Input: 
            lis[int] group
        Output: 
            float entropy
        '''
        # count numbers of each class
        count = Counter(group)
        # compute the percentage of each class
        prob = np.array(list(count.values()))/len(group)
        
        # compute entropy
        return -np.sum(prob * np.log2(prob))
        
        
    def info_gain(self, parent, left, right):
        '''
        Compute information gain on each node
        Input:
            list parent
            list left
            list right
        Output:
            float information gain
        '''
        left_portion = len(left) / len(parent)
        right_portion = len(right) / len(right)
        
        return self.entropy(parent) - (left_portion * self.entropy(left) +
                                      right_portion * self.entropy(right))
        

        
    def best_split(self, X, Y):
        '''
        Choose the best feature to split based on information gain
        Input:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
        Output:
            dictionary best_split
        '''
        
        best_split = {}
        best_info_gain =  -1
        n_row, n_col = X.shape
        
        if self.feature_bagging:
            _feature = np.random.choice(n_col, self.feature_bagging, replace = False)
        else:
            _feature = range(n_col)

        # for every feature in X
        for f_idx in _feature:
            X_f = X[:, f_idx]
            # for unique value of the choosen feature
            for threshold in np.unique(X_f):
                # take threshold for split the data
                # might be optimized by using bins of the unique value
                # instead of all of them
                # split based on threshold, left mean <= threshold
                X_left = X[X_f <= threshold,:]
                X_right = X[X_f > threshold,:]
                
                
                if len(X_left) > 0 and len(X_right) > 0:
                    Y_left = Y[X_f <= threshold]
                    Y_right = Y[X_f > threshold]
                    gain = self.info_gain(Y, Y_left, Y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'X_left': X_left,
                            'X_right': X_right,
                            'Y_left': Y_left,
                            'Y_right': Y_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
                    
                
        
    def grow(self, X, Y, depth = 0):
        '''
        Build the tree
        Input:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
            int depth: current depth of a tree
        Output:
            Node root
        '''
        n_row, n_col = X.shape
        # check for pre-pruning conditions
        if n_row >= self.min_samples_split and depth <= self.max_depth:
            best = self.best_split(X,Y)
            # check if the split is pure
            if best['gain'] > 0:
                # grow a tree on left and right
                left = self.grow(X=best['X_left'],
                                Y=best['Y_left'],
                                depth = depth+1)
                right = self.grow(X=best['X_right'],
                                Y=best['Y_right'],
                                depth = depth+1)
                return Node(feature=best['feature_index'],
                           threshold=best['threshold'],
                           left=left,
                           right=right,
                           gain=best['gain'])
        # stop/leaf node, return the most common target value
        return Node(value=Counter(Y).most_common(1)[0][0])     
        
    def fit(self, X, Y):
        '''
        Train the decision tree with data
        Input:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
        Output:
            None
        '''
        self.root = self.grow(X,Y)
        
    def _predict(self, x, tree):
        '''
        Predict a single sample based on a given tree
        Input:
            np.array X: shape (1, p)
            Tree tree
        Output:
           float prediction
        '''
        
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # go downwards
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.left)
        elif feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.right)
        
    def predict(self, X):
        '''
        Predict a single sample based on a given tree
        Input:
            np.array X: shape (n, p)
        Output:
           np.array prediction: shape (n)
        '''
        return np.apply_along_axis(self._predict, -1, X, self.root)

## Test classification tree with iris dataset

In [351]:
from sklearn.datasets import load_iris

data = load_iris()
X = data['data']
y = data['target']

In [352]:
X.shape, y.shape

((150, 4), (150,))

In [356]:
clf = Tree(feature_bagging=2)

In [357]:
clf.fit(X, y)

In [358]:
print('Train accuracy is {}'.format(sum(clf.predict(X) == y)/len(y)))

Train accuracy is 0.9733333333333334


Looks like we have maken the tree work.

In [359]:
# Implement random forest

class RandomForest:
    '''
    Implement random forest algorith based on class Tree
    '''
    def __init__(self, num_trees=30, min_samples_split=5, 
                 max_depth=5, feature_bagging = None):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.trees = [] # collection of generated trees
        self.feature_bagging = feature_bagging
        
    def bootstrap(self, X, Y):
        '''
        Sample data with replacement
        Input:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
        Output:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
        '''
        
        n_row, n_col = X.shape
        sample = np.random.choice(n_row, n_row, replace = True)
        return X[sample], Y[sample]
    
    def fit(self, X, Y):
        '''
        Train a random forest model based on data
        Input:
            np.array X: shape (n, p)
            np.array Y: shape (n,)
        Output:
            None
        '''
        
        tree_count = 0
        while tree_count < self.num_trees:
            try:
                clf = Tree(min_samples_split=self.min_samples_split,
                          max_depth=self.max_depth,
                          feature_bagging=self.feature_bagging)
                _X, _Y = self.bootstrap(X, Y)
                clf.fit(_X, _Y)
                self.trees.append(clf)
                tree_count += 1
            except Exception as e:
                continue
                
    def predict(self, X):
        '''
        Predict a single sample based on a given random forest
        Input:
            np.array X: shape (n, p)
        Output:
           np.array prediction: shape (n)
        '''
        y_hat = np.array(list(map(lambda x: x.predict(X), self.trees)))
        
        pred, _ = mode(y_hat, axis = 0)
        
        return pred.ravel()

## Test

In [360]:
data = load_iris()
X = data['data']
y = data['target']

In [364]:
clf = RandomForest(feature_bagging=3)

In [365]:
clf.fit(X, y)

In [366]:
print('Train accuracy is {}'.format(sum(clf.predict(X) == y)/len(y)))

Train accuracy is 0.9866666666666667


## The next step is to optimize runtime and apply parallelization when growing trees in random forest.

I will use `joblib` packages to do parallel processing.

In [367]:
from joblib import Parallel

In [368]:
Parallel(n_jobs=-2)

Parallel(n_jobs=-2)